In [1]:
# Setting stuff up
# Select and install requirements
reqs = """
pytest-playwright==0.4.3
"""

with open(r'requirements.txt', 'w') as reqs_txt:
    reqs_txt.write(reqs)
    reqs_txt.close()

! pip install -r requirements.txt

! playwright install

In [2]:
import re
import playwright
from playwright.sync_api import Page, expect, sync_playwright

with sync_playwright() as pw:
    # create browser instance
    browser = pw.chromium.launch(
        # we can choose either a Headful (With GUI) or Headless mode:
        headless=False,
    )
    # create context
    # using context we can define page properties like viewport dimensions
    context = browser.new_context(
        # most common desktop viewport is 1920x1080
        viewport={"width": 1920, "height": 1080}
    )
    # create page aka browser tab which we'll be using to do everything
    page = context.new_page()
    page.goto("https://search.ltn.com.tw/list?keyword=的&start_time=20130101&end_time=20130102&sort=date&type=all&page=1")

Error: It looks like you are using Playwright Sync API inside the asyncio loop.
Please use the Async API instead.

In [ ]:
import asyncio
from playwright.async_api import async_playwright

# Function to scrape a single page
async def scrape_page(page):
    items = []
    # Replace 'selector_for_list_item' with the actual selector of the list items you want to scrape
    list_items = await page.query_selector_all('selector_for_list_item')
    for item in list_items:
        # Extract the text or any other attribute you're interested in
        text = await item.text_content()
        items.append(text)
    return items

# Function to handle pagination and scrape all pages
async def scrape_all_pages():
    # Start Playwright in asynchronous mode
    async with async_playwright() as p:
        browser = await p.chromium.launch()
        page = await browser.new_page()
        await page.goto('https://search.ltn.com.tw/list?keyword=的&start_time=20130101&end_time=20130102&sort=date&type=all&page=1') 

        all_items = []

        while True:
            # Scrape the current page
            current_page_items = await scrape_page(page)
            all_items.extend(current_page_items)

            # Try to find the next button and click it to go to the next page
            # Replace 'selector_for_next_button' with the actual selector for the pagination next button
            next_button = await page.query_selector('selector_for_next_button')
            if next_button and await next_button.is_visible():
                await next_button.click()
                # Wait for navigation to complete before starting the next scrape
                await page.wait_for_load_state('networkidle')
            else:
                # If there's no next button, we've reached the last page
                break

        await browser.close()
        return all_items

# Running the async function in Jupyter Notebook
%pip install nest_asyncio
import nest_asyncio
nest_asyncio.apply()

# Run the scraping
results = await scrape_all_pages()

# Now, results holds all the scraped items
print(results)
